In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import LinearNDInterpolator

from glob import glob
from tqdm import tqdm
import pandas as pd
import os

import astropy.units as u
from astropy.constants import M_jup, M_sun
from astropy.time import Time
from astropy.table import Table, join
from astropy.coordinates import SkyCoord, search_around_sky

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

In [ ]:
planets = NasaExoplanetArchive.query_criteria(table='pscomppars', cache=True)
planets.add_index('hostname')

In [ ]:
owls_sindices = pd.read_pickle('data/db.pkl')

In [ ]:
url = (
    'https://docs.google.com/spreadsheets/d/'
    '11Z7B76FXBkEwcGmhp72sC6AQdP8ER8K_eU5RAW8ed2M'
    '/gviz/tq?tqx=out:csv&sheet=catalog'
)
owls_sheet = pd.read_csv(url)

In [ ]:
planet_coords = SkyCoord(ra=planets['ra'], dec=planets['dec'])
owls_coords = SkyCoord(ra=owls_sheet['ra'], dec=owls_sheet['dec'], unit=(u.hourangle, u.deg))

owls_cols = list(owls_sheet.columns)
idx1, idx2 = search_around_sky(owls_coords, planet_coords, 2*u.arcmin)[:2]

planets_cols = list(pd.Series(planets.colnames).drop_duplicates().values) #list(planets.colnames) + ['pl_hostname']

# Find planets that have matching OWLS coordinates within 5 arcsec
owls_sheet_reind = owls_sheet[owls_cols]
owls_sheet_reind.index = np.arange(len(owls_sheet_reind))

owls_planets = pd.merge(
    owls_sheet_reind, planets[idx2][planets_cols].to_pandas(), left_on='pl_hostname', right_on='hostname'# levels=['pl_hostname', 'pl_name']
)
owls_planets.index = owls_planets['pl_letter']
groups = dict()
for name, group in owls_planets.groupby('pl_hostname'):
    groups[name] = group.reindex(index=list(group['pl_letter']))
owls_planets = pd.concat(groups)

In [ ]:
planet_coords = SkyCoord(ra=planets['ra'], dec=planets['dec'])
owls_coords = SkyCoord(ra=owls_sheet['ra'], dec=owls_sheet['dec'], unit=(u.hourangle, u.deg))

In [ ]:
idx1, idx2 = search_around_sky(owls_coords, planet_coords, 2*u.arcmin)[:2]

In [ ]:
owls_cols = list(owls_sheet.columns)

planets_cols = list(pd.Series(planets.colnames).drop_duplicates().values) #list(planets.colnames) + ['pl_hostname']
drop_planets_keys = list(set(owls_sheet_reind.columns).intersection(set(planets.colnames)))
for k in drop_planets_keys:
    planets_cols.pop(planets_cols.index(k))

In [ ]:
# Find planets that have matching OWLS coordinates within 5 arcsec
owls_sheet_reind = owls_sheet[owls_cols] # .iloc[idx1]
owls_sheet_reind.index = np.arange(len(owls_sheet_reind)) #planets[idx2]['pl_name'].value

owls_planets = pd.merge(
    owls_sheet_reind, planets[idx2][planets_cols].to_pandas(), left_on='pl_hostname', right_on='hostname'# levels=['pl_hostname', 'pl_name']
)
owls_planets.index = owls_planets['pl_letter']
groups = dict()
for name, group in owls_planets.groupby('pl_hostname'):
    groups[name] = group.reindex(index=list(group['pl_letter']))
owls_planets = pd.concat(groups)

In [ ]:
short_prot_flag = (
    (~np.isnan(owls_planets['st_rotp']) & 
     (owls_planets['st_rotp'] < 10)) | 
    ((owls_planets['Prot_phot_days'] < 10) & 
     (owls_planets['Prot_phot_days'] > 0))
)

short_prot_flag_planet_bs = (
    (~np.isnan(owls_planets[owls_planets['pl_letter'] == 'b']['st_rotp']) & 
     (owls_planets[owls_planets['pl_letter'] == 'b']['st_rotp'] < 10)) | 
    ((owls_planets[owls_planets['pl_letter'] == 'b']['Prot_phot_days'] < 10) & 
     (owls_planets[owls_planets['pl_letter'] == 'b']['Prot_phot_days'] > 0))
)

In [ ]:
vansaders2012 = Table.read("data/vanSaders2012.txt", format='ascii')

In [ ]:
tmp = vansaders2012[['Mass', 'Age', 'Teff']]

In [ ]:
arr = vansaders2012[['Mass', 'Age', 'Teff']]
arr.sort("Mass")
arr = arr.as_array()

# make uniform ages for each target by taking mean of all available refs
for target, group in owls_planets.groupby('hostname'):
    owls_planets.loc[target, 'st_age'] = group['st_age'].mean()

In [ ]:
ncols = 32
dtypes = ncols * [float,]
dtypes[0] = "U16"
dtypes[-1] = "U16"

colnames = "SpT   Teff  logT   BCv    logL   Mbol R_Rsun   Mv    B-V    Bt-Vt  G-V    Bp-Rp  G-Rp   M_G    b-y    U-B    V-Rc   V-Ic   V-Ks   J-H    H-Ks   M_J    M_Ks   Ks-W1   W1-W2  W1-W3  W1-W4   g-r   i-z  z-Y  Msun SpT2".split()

mamajek = Table(np.genfromtxt("data/EEM_dwarf_UBVIJHK_colors_Teff.txt", dtype=dtypes), names=colnames)
mamajek.add_index("SpT")
estimated_owls_teffs = mamajek.loc[[s if not s.endswith('e') else s[:-1] for s in owls_planets['sp_type_mamajek']]]['Teff'].data
estimated_owls_mass = mamajek.loc[[s if not s.endswith('e') else s[:-1] for s in owls_planets['sp_type_mamajek']]]['Msun'].data

owls_planets['st_teff_mamajek'] = estimated_owls_teffs
owls_planets['st_mass_mamajek'] = estimated_owls_mass

In [ ]:
interp = LinearNDInterpolator(
    np.vstack(arr.tolist()), vansaders2012['Rcz/R'].data, rescale=True, 
    # fill the rest with unity since they're likely fully convective stars
    fill_value=1
)

epsilons = []
for i, table in enumerate([planets.to_pandas(), owls_planets]):
    # feh = owls_planets['st_met'].values
    st_mass = table['st_mass'].values
    st_age = table['st_age'].values
    st_teff = table['st_teff'].values
    
    if i == 0:
        st_mass[np.isnan(st_mass)] = 1
        st_age[np.isnan(st_age)] = 10
        st_teff[np.isnan(st_teff)] = 5700

    else:
        st_mass[np.isnan(st_mass)] = owls_planets['st_mass_mamajek'].iloc[np.isnan(st_mass)]
        st_age[np.isnan(st_age)] = 10
        st_teff[np.isnan(st_teff)] = owls_planets['st_teff_mamajek'].iloc[np.isnan(st_teff)]
        
    Rcz = interp(st_mass, st_age, st_teff)
    rad_unit = float(1 * u.R_sun / u.AU)
    mass_unit = float(1 * u.M_jup / u.M_sun)
    planet_mass = table['pl_bmassj']
    stellar_radius = table['st_rad']
    smax = table['pl_orbsmax']
    
    epsilon = (
        (mass_unit * planet_mass / st_mass) * 
        (rad_unit * Rcz * stellar_radius / smax) ** 3
    )
    # print(epsilon)

    epsilons.append(pd.Series(epsilon, table.index))

In [ ]:
plt.hist(np.log10(epsilons[0]), 30, log=True, label='All known planets');
plt.hist(np.log10(epsilons[1]), 30, log=True, label='OWLS planets');
plt.gca().set(
    xlabel='$\\log_{10} \\varepsilon$', 
    ylabel='Targets',
    title='$\\varepsilon \\rightarrow 1$ = stronger tides'
)
plt.legend()
plt.savefig('plots/epsilon.pdf')